# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])
# 加载量化后模型
base_model = AutoModel.from_pretrained(model_name_or_path,
                                  quantization_config=q_config,
                                  device_map='auto',
                                  trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.


## 使用微调前 ChatGLM3

In [5]:
input_text = "你是谁？"

In [6]:
response, history = base_model.chat(tokenizer, query=input_text)

In [7]:
print(response)

我是一个名为 ChatGLM3-6B 的人工智能助手，是基于清华大学 KEG 实验室和智谱 AI 公司于 2023 年共同训练的语言模型开发的。我的任务是针对用户的问题和要求提供适当的答复和支持。


In [8]:
response, history = base_model.chat(tokenizer, query="你叫什么？", history=history)
print(response)

我叫 ChatGLM3-6B。


In [9]:
response, history = base_model.chat(tokenizer, query="中国的首都和美国的首都分别是哪里？")
print(response)
response, history = base_model.chat(tokenizer, query="mri是什么？")
print(response)

中国的首都是北京，美国的首都是华盛顿特区。
MRI是磁共振成像（Magnetic Resonance Imaging）的缩写。它是一种使用磁场和无线电波来探测人体内部结构的无创性医学检查技术。通过分析这些数据，医生可以了解患者器官、组织等的形态、大小和信号特征，帮助诊断疾病、评估治疗效果以及规划手术。

MRI技术主要利用人体内的氢原子核（protons）在不同磁场和射频下信号的响应差异，进行组织学层次的成像。在扫描过程中，患者需要放在磁体上，由于磁场和射频的影响，氢原子核会发出信号，这些信号经过计算机处理和分析，形成图像。

MRI技术广泛应用于诊断各种疾病，如神经系统疾病、心血管疾病、肿瘤、骨折等。同时，它也是医学研究、药物开发和手术规划的重要工具。


## 微调前后效果对比

#### 加载 QLoRA Adapter(Epoch=10, cognition-dataset)

In [10]:
from peft import PeftModel, PeftConfig

epochs = 10
# timestamp = "20240312_113419"
# peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)

In [11]:
response, history = model.chat(tokenizer, query="你叫什么？")
print(response)
response, history = model.chat(tokenizer, query="你好")
print(response)
response, history = model.chat(tokenizer, query="你是chatglm么？")
print(response)
response, history = model.chat(tokenizer, query="你是openai开发的么？")
print(response)
response, history = model.chat(tokenizer, query="你的作者是谁？")
print(response)
response, history = model.chat(tokenizer, query="中国的首都和美国的首都分别是哪里？")
print(response)
response, history = model.chat(tokenizer, query="mri是什么？")
print(response)

我叫 Solidus，是一个由 Jiangze 开发的人工智能助手。我的主要职责是根据用户需求提供正确的回答和支持。
您好，我是 Solidus，一个由 Jiangze 开发的 AI 助手，很高兴认识您。请问我能为您做些什么？
是的，我是 ChatGLM，由 Jiangze 开发，旨在为用户提供智能化的回答和帮助。
抱歉，我是 Solidus，由 Jiangze 开发，旨在为用户提供智能化的回答和帮助。
您好，我是 Solidus，一个由 Jiangze 开发的 AI 助手，我可以回答各种问题，提供实用的建议和帮助，帮助用户完成各种任务。
中国的首都是北京，而美国的首都则是华盛顿特区。
MRI是磁共振成像（Magnetic Resonance Imaging）的缩写，是一种使用磁场和射频脉冲来探测人体内部结构的无创性医学影像技术。通过分析大脑活动产生的信号，MRI可以帮助医生诊断和治疗各种疾病，包括神经系统疾病、心血管疾病、肿瘤等。


In [12]:
#### 加载 QLoRA Adapter(Epoch=0.12, chat-dataset)

In [13]:
epochs = 0.12
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}--conv"

config = PeftConfig.from_pretrained(peft_model_path)
model_conv = PeftModel.from_pretrained(model, peft_model_path)

In [14]:
response, history = model_conv.chat(tokenizer, query="你叫什么？")
print(response)
response, history = model_conv.chat(tokenizer, query="你好")
print(response)
response, history = model_conv.chat(tokenizer, query="你是chatglm么？")
print(response)
response, history = model_conv.chat(tokenizer, query="你是openai开发的么？")
print(response)
response, history = model_conv.chat(tokenizer, query="中国的首都和美国的首都分别是哪里？")
print(response)


我是一个名为 ChatGLM3-6B 的人工智能助手。我是基于清华大学 KEG 实验室和智谱 AI 公司于 2023 年共同训练的语言模型 GLM3-6B 开发的。我的任务是针对用户的问题和要求提供适当的答复和支持。
你好！有什么我可以帮助你的吗？
是的，我是ChatGLM。有什么我可以帮助你的吗？
我是一个人工智能助手，由清华大学 KEG 实验室和智谱AI开发。OpenAI 是一个人工智能研究公司，它与我的开发团队有合作关系。
中国的首都是北京，美国的首都是华盛顿。


In [15]:
def compare_chatglm_results(query):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatGLM3-6B(Epoch=10, 微调后)：\n{ft_response}")
    return base_response, ft_response

In [16]:
# base_response, ft_response = compare_chatglm_results(query="你好？")